# Módulo 2: Prompt Engineering Avanzado para Desarrollo
## Integración en Sistemas Backend y Desarrollo de Agentes/Workflows

**Duración estimada**: 1 hora  
**Nivel**: Intermedio  
**Objetivo**: Aplicar prompt engineering avanzado en desarrollo backend y sistemas

---

## 📚 Contenido del Módulo

Este notebook cubre:
1. **Práctica 2.1**: Sistema de agentes con múltiples pasos
2. **Práctica 2.2**: Function calling para integración con APIs
3. **Práctica 2.3**: Workflow complejo con validación y manejo de errores

---

## 🎯 Objetivos de Aprendizaje

Al finalizar este módulo serás capaz de:
- Diseñar y construir agentes autónomos que ejecuten tareas complejas
- Integrar IA con sistemas externos usando function calling
- Crear workflows robustos con múltiples llamadas coordinadas
- Implementar validación y manejo de errores en sistemas con IA
- Optimizar costos y latencia en aplicaciones de producción

---

## ⚙️ Configuración Inicial

Antes de comenzar, asegúrate de tener:
- Python 3.8+ instalado
- Cuenta de OpenAI con API key
- Librerías necesarias instaladas

**Instalación en Windows:**
```bash
pip install openai python-dotenv requests
```

In [ ]:
# Instalación de dependencias (ejecutar solo si es necesario)
# !pip install openai python-dotenv requests

# Importar librerías necesarias
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
import requests
from typing import Dict, List, Any

# Cargar variables de entorno
load_dotenv()

# Inicializar cliente de OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Verificar configuración
if not client.api_key:
    print("⚠️ ADVERTENCIA: No se encontró OPENAI_API_KEY. Configúrala en un archivo .env")
else:
    print("✅ Cliente de OpenAI configurado correctamente")

---

# Práctica 2.1: Sistema de Agentes con Múltiples Pasos

## 🎯 Objetivo
Crear un agente autónomo que ejecute tareas complejas descomponiéndolas en múltiples pasos, coordinando varias llamadas a la IA de manera secuencial.

## 📖 Contexto y Teoría

### ¿Qué es un Agente de IA?

Un agente es un sistema que puede:
- **Planificar**: Descomponer tareas complejas en pasos
- **Ejecutar**: Realizar acciones en secuencia
- **Decidir**: Elegir el siguiente paso basado en resultados anteriores
- **Iterar**: Ajustar su comportamiento según feedback

### Arquitectura de un Agente Multi-Paso

```
1. Análisis inicial → Entender la tarea
2. Planificación → Descomponer en pasos
3. Ejecución → Ejecutar cada paso secuencialmente
4. Validación → Verificar resultados
5. Síntesis → Combinar resultados finales
```

### Casos de Uso
- Análisis de código multi-etapa
- Generación de documentación completa
- Code review automatizado
- Refactorización asistida

## 🔍 Ejercicio Guiado: Agente de Análisis de Código Multi-Etapa

Vamos a crear un agente que analice código, genere tests y documentación en pasos coordinados.

In [ ]:
# EJEMPLO: Agente que analiza código en múltiples pasos

def agente_analisis_codigo(codigo: str) -> Dict[str, Any]:
    """
    Agente que analiza código en múltiples etapas:
    1. Análisis del código
    2. Generación de tests
    3. Generación de documentación
    """
    
    resultados = {}
    
    # PASO 1: Análisis del código
    print("🔍 Paso 1: Analizando código...")
    prompt_analisis = f"""
Analiza el siguiente código Python y proporciona:
1. Propósito de la función/clase
2. Complejidad temporal y espacial
3. Posibles bugs o mejoras
4. Dependencias y requisitos

Código:
```python
{codigo}
```

Responde en formato JSON con las claves: proposito, complejidad, mejoras, dependencias
"""
    
    response1 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_analisis}],
        temperature=0.3,
        response_format={"type": "json_object"}
    )
    
    analisis = json.loads(response1.choices[0].message.content)
    resultados['analisis'] = analisis
    print(f"✅ Análisis completado: {analisis.get('proposito', 'N/A')[:50]}...")
    
    # PASO 2: Generación de tests basado en el análisis
    print("\n🧪 Paso 2: Generando tests...")
    prompt_tests = f"""
Basándote en este análisis:
{json.dumps(analisis, indent=2, ensure_ascii=False)}

Y este código:
```python
{codigo}
```

Genera tests unitarios completos usando pytest. Incluye:
- Tests de casos normales
- Tests de casos edge
- Tests de manejo de errores

Retorna SOLO el código Python de los tests, sin explicaciones.
"""
    
    response2 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_tests}],
        temperature=0.3
    )
    
    tests = response2.choices[0].message.content
    resultados['tests'] = tests
    print("✅ Tests generados")
    
    # PASO 3: Generación de documentación
    print("\n📝 Paso 3: Generando documentación...")
    prompt_docs = f"""
Basándote en este análisis:
{json.dumps(analisis, indent=2, ensure_ascii=False)}

Y este código:
```python
{codigo}
```

Genera documentación completa en formato Markdown que incluya:
- Descripción general
- Parámetros y retornos
- Ejemplos de uso
- Notas sobre complejidad y limitaciones

Retorna SOLO la documentación en Markdown.
"""
    
    response3 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_docs}],
        temperature=0.3
    )
    
    documentacion = response3.choices[0].message.content
    resultados['documentacion'] = documentacion
    print("✅ Documentación generada")
    
    return resultados

# Código de ejemplo para analizar
codigo_ejemplo = """
def buscar_binaria(lista, objetivo):
    izquierda, derecha = 0, len(lista) - 1
    
    while izquierda <= derecha:
        medio = (izquierda + derecha) // 2
        
        if lista[medio] == objetivo:
            return medio
        elif lista[medio] < objetivo:
            izquierda = medio + 1
        else:
            derecha = medio - 1
    
    return -1
"""

# Ejecutar el agente
print("=" * 60)
print("AGENTE DE ANÁLISIS DE CÓDIGO MULTI-ETAPA")
print("=" * 60 + "\n")

resultados = agente_analisis_codigo(codigo_ejemplo)

# Mostrar resultados
print("\n" + "=" * 60)
print("RESULTADOS FINALES")
print("=" * 60)
print(f"\n📊 Análisis:\n{json.dumps(resultados['analisis'], indent=2, ensure_ascii=False)}")
print(f"\n🧪 Tests:\n{resultados['tests']}")
print(f"\n📝 Documentación:\n{resultados['documentacion']}")

In [ ]:
# ESPACIO PARA TU CÓDIGO
def agente_code_review(codigo: str) -> Dict[str, Any]:
    """
    Agente que realiza code review en múltiples pasos
    """
    # TODO: Implementa los 4 pasos del code review
    # Paso 1: Detectar problemas potenciales
    # Paso 2: Verificar mejores prácticas
    # Paso 3: Sugerir mejoras con ejemplos
    # Paso 4: Generar reporte consolidado
    
    resultados = {}
    
    # Tu implementación aquí
    
    return resultados

# Código para revisar
codigo_revisar = """
def procesar_usuarios(users):
    result = []
    for u in users:
        if u['age'] > 18:
            if u['status'] == 'active':
                result.append(u['name'].upper())
    return result
"""

# Ejecuta el agente
# resultados_review = agente_code_review(codigo_revisar)
# print(json.dumps(resultados_review, indent=2, ensure_ascii=False))

## 🔥 Ejercicio de Desafío 2.1 (Opcional)

Crea un agente de refactorización inteligente que:
1. Analice código legacy
2. Identifique patrones a mejorar
3. Proponga refactorizaciones específicas
4. Genere código refactorizado
5. Compare antes/después con métricas (complejidad, líneas, etc.)

**Desafío extra**: Haz que el agente pueda iterar sobre sus propias sugerencias y mejorarlas.

---

# Práctica 2.2: Function Calling para Integración con APIs

## 🎯 Objetivo
Aprender a usar function calling (tool use) para conectar modelos de IA con herramientas externas, APIs y sistemas.

## 📖 Contexto y Teoría

### ¿Qué es Function Calling?

Function calling permite que los modelos de IA:
- **Llamen funciones**: Invocar código Python o APIs externas
- **Obtengan datos**: Acceder a información en tiempo real
- **Interactúen con sistemas**: Conectar con bases de datos, servicios, etc.
- **Extender capacidades**: Ir más allá de lo que el modelo sabe

### Flujo de Function Calling

```
1. Usuario hace pregunta
2. Modelo decide qué función necesita
3. Sistema ejecuta la función
4. Resultado se pasa de vuelta al modelo
5. Modelo genera respuesta final usando los datos
```

### Casos de Uso
- Consultas a bases de datos
- Búsquedas en APIs externas
- Cálculos complejos
- Acceso a información en tiempo real
- Integración con sistemas legacy

## 🔍 Ejercicio Guiado: Agente con Function Calling

Vamos a crear un agente que puede consultar APIs externas y realizar cálculos.

In [ ]:
# EJEMPLO: Agente con function calling para consultar APIs

# Definimos las funciones disponibles para el modelo
def obtener_clima(ciudad: str) -> str:
    """
    Obtiene el clima actual de una ciudad (simulado)
    En producción, esto llamaría a una API real como OpenWeatherMap
    """
    # Simulación de datos de clima
    datos_clima = {
        "Buenos Aires": "25°C, soleado",
        "Madrid": "18°C, nublado",
        "Nueva York": "12°C, lluvioso",
        "Tokio": "22°C, parcialmente nublado"
    }
    return datos_clima.get(ciudad, f"Clima no disponible para {ciudad}")

def calcular_estadisticas(numeros: List[float]) -> Dict[str, float]:
    """
    Calcula estadísticas básicas de una lista de números
    """
    if not numeros:
        return {"error": "Lista vacía"}
    
    return {
        "suma": sum(numeros),
        "promedio": sum(numeros) / len(numeros),
        "minimo": min(numeros),
        "maximo": max(numeros),
        "cantidad": len(numeros)
    }

def buscar_informacion(tema: str) -> str:
    """
    Busca información sobre un tema (simulado)
    En producción, esto podría buscar en una base de datos o API
    """
    # Simulación de búsqueda
    informacion = {
        "python": "Python es un lenguaje de programación de alto nivel, interpretado y de propósito general.",
        "openai": "OpenAI es una empresa de investigación en inteligencia artificial.",
        "api": "Una API (Application Programming Interface) es un conjunto de protocolos y herramientas para construir software."
    }
    return informacion.get(tema.lower(), f"Información no disponible sobre {tema}")

# Definimos las funciones para el modelo
funciones_disponibles = [
    {
        "type": "function",
        "function": {
            "name": "obtener_clima",
            "description": "Obtiene el clima actual de una ciudad",
            "parameters": {
                "type": "object",
                "properties": {
                    "ciudad": {
                        "type": "string",
                        "description": "Nombre de la ciudad"
                    }
                },
                "required": ["ciudad"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calcular_estadisticas",
            "description": "Calcula estadísticas (suma, promedio, min, max) de una lista de números",
            "parameters": {
                "type": "object",
                "properties": {
                    "numeros": {
                        "type": "array",
                        "items": {"type": "number"},
                        "description": "Lista de números para analizar"
                    }
                },
                "required": ["numeros"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "buscar_informacion",
            "description": "Busca información sobre un tema específico",
            "parameters": {
                "type": "object",
                "properties": {
                    "tema": {
                        "type": "string",
                        "description": "Tema sobre el cual buscar información"
                    }
                },
                "required": ["tema"]
            }
        }
    }
]

# Mapeo de nombres de funciones a funciones reales
funciones_reales = {
    "obtener_clima": obtener_clima,
    "calcular_estadisticas": calcular_estadisticas,
    "buscar_informacion": buscar_informacion
}

def ejecutar_agente_con_funciones(mensaje: str) -> str:
    """
    Ejecuta un agente que puede usar function calling
    """
    mensajes = [{"role": "user", "content": mensaje}]
    
    # Primera llamada: el modelo decide si necesita funciones
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=mensajes,
        tools=[{"type": "function", "function": f["function"]} for f in funciones_disponibles],
        tool_choice="auto"
    )
    
    mensaje_respuesta = response.choices[0].message
    
    # Si el modelo quiere usar funciones, las ejecutamos
    if mensaje_respuesta.tool_calls:
        # Agregamos la respuesta del modelo al historial
        mensajes.append(mensaje_respuesta)
        
        # Ejecutamos cada función solicitada
        for tool_call in mensaje_respuesta.tool_calls:
            nombre_funcion = tool_call.function.name
            argumentos = json.loads(tool_call.function.arguments)
            
            # Ejecutamos la función real
            funcion = funciones_reales[nombre_funcion]
            resultado = funcion(**argumentos)
            
            # Agregamos el resultado al historial
            mensajes.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": nombre_funcion,
                "content": json.dumps(resultado, ensure_ascii=False)
            })
        
        # Segunda llamada: el modelo genera respuesta final usando los resultados
        response_final = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=mensajes
        )
        
        return response_final.choices[0].message.content
    else:
        # Si no necesita funciones, retorna la respuesta directa
        return mensaje_respuesta.content

# Probar el agente
print("=" * 60)
print("AGENTE CON FUNCTION CALLING")
print("=" * 60 + "\n")

preguntas = [
    "¿Cuál es el clima en Buenos Aires?",
    "Calcula las estadísticas de estos números: [10, 20, 30, 40, 50]",
    "¿Qué es Python?",
    "Dame el clima en Madrid y calcula estadísticas de [5, 15, 25, 35]"
]

for pregunta in preguntas:
    print(f"❓ Pregunta: {pregunta}")
    respuesta = ejecutar_agente_con_funciones(pregunta)
    print(f"💬 Respuesta: {respuesta}\n")
    print("-" * 60 + "\n")

In [ ]:
# ESPACIO PARA TU CÓDIGO
# Base de datos simulada de usuarios
base_datos_usuarios = [
    {"id": 1, "nombre": "Juan Pérez", "email": "juan@example.com", "activo": True},
    {"id": 2, "nombre": "María García", "email": "maria@example.com", "activo": True},
    {"id": 3, "nombre": "Carlos López", "email": "carlos@example.com", "activo": False},
]

# TODO: Implementa las funciones
def buscar_usuario(id: int) -> Dict:
    """Busca un usuario por ID"""
    # Tu código aquí
    pass

def listar_usuarios(limite: int = 10) -> List[Dict]:
    """Lista usuarios con un límite"""
    # Tu código aquí
    pass

def crear_usuario(nombre: str, email: str) -> Dict:
    """Crea un nuevo usuario"""
    # Tu código aquí
    pass

# TODO: Define las funciones para function calling
funciones_usuarios = [
    # Define las funciones aquí siguiendo el formato del ejemplo
]

# TODO: Crea el mapeo de funciones
funciones_reales_usuarios = {
    # Mapea nombres a funciones reales
}

# TODO: Implementa el agente
def agente_usuarios(pregunta: str) -> str:
    """Agente que puede consultar y gestionar usuarios"""
    # Tu implementación aquí
    pass

# Prueba el agente
# print(agente_usuarios("¿Cuántos usuarios hay?"))
# print(agente_usuarios("Muéstrame el usuario con ID 1"))
# print(agente_usuarios("Crea un usuario llamado Ana con email ana@example.com"))

## 🔥 Ejercicio de Desafío 2.2 (Opcional)

Crea un agente que integre múltiples APIs externas:
- API de conversión de monedas (simulada)
- API de noticias (simulada)
- Calculadora financiera

El agente debe poder responder preguntas complejas como:
- "¿Cuánto son 100 USD en EUR y qué noticias hay sobre economía?"
- "Calcula el interés compuesto de $1000 al 5% anual por 3 años"

**Desafío extra**: Implementa caché para evitar llamadas repetidas a las APIs.

---

# Práctica 2.3: Workflow Complejo con Validación y Manejo de Errores

## 🎯 Objetivo
Construir un pipeline robusto con múltiples llamadas a IA, incluyendo validación, manejo de errores y optimización.

## 📖 Contexto y Teoría

### ¿Qué es un Workflow Complejo?

Un workflow es una secuencia de pasos que:
- **Coordina múltiples llamadas**: Diferentes modelos o funciones
- **Maneja errores**: Recuperación y fallbacks
- **Valida resultados**: Verifica calidad antes de continuar
- **Optimiza recursos**: Minimiza costos y latencia

### Componentes de un Workflow Robusto

1. **Validación de entrada**: Verificar que los datos son correctos
2. **Ejecución con retry**: Reintentar en caso de fallos
3. **Validación de salida**: Verificar que los resultados son válidos
4. **Manejo de errores**: Fallbacks y mensajes claros
5. **Logging**: Registrar cada paso para debugging

### Casos de Uso
- Pipelines de procesamiento de datos
- Sistemas de análisis automatizado
- Generación de contenido multi-etapa
- Sistemas de decisión automatizados

## 🔍 Ejercicio Guiado: Sistema de Análisis de Código con Validación

Vamos a crear un workflow que analice código con múltiples etapas y validaciones.

In [ ]:
# EJEMPLO: Workflow robusto con validación y manejo de errores

import time
from typing import Optional

def validar_codigo_python(codigo: str) -> tuple[bool, Optional[str]]:
    """
    Valida que el código sea Python válido
    Retorna: (es_valido, mensaje_error)
    """
    try:
        compile(codigo, '<string>', 'exec')
        return True, None
    except SyntaxError as e:
        return False, f"Error de sintaxis: {str(e)}"
    except Exception as e:
        return False, f"Error: {str(e)}"

def llamada_ia_con_retry(prompt: str, max_intentos: int = 3, delay: float = 1.0) -> Optional[str]:
    """
    Realiza una llamada a la IA con reintentos en caso de error
    """
    for intento in range(max_intentos):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                timeout=30  # Timeout de 30 segundos
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"⚠️ Intento {intento + 1} falló: {str(e)}")
            if intento < max_intentos - 1:
                time.sleep(delay * (intento + 1))  # Backoff exponencial
            else:
                print(f"❌ Todos los intentos fallaron")
                return None
    return None

def validar_respuesta_json(respuesta: str) -> tuple[bool, Optional[Dict]]:
    """
    Valida que la respuesta sea JSON válido
    """
    try:
        datos = json.loads(respuesta)
        return True, datos
    except json.JSONDecodeError:
        # Intentar extraer JSON del texto
        import re
        json_match = re.search(r'\{.*\}', respuesta, re.DOTALL)
        if json_match:
            try:
                datos = json.loads(json_match.group())
                return True, datos
            except:
                pass
        return False, None

def workflow_analisis_codigo(codigo: str) -> Dict[str, Any]:
    """
    Workflow completo de análisis de código con validaciones
    """
    resultado = {
        "exitoso": False,
        "errores": [],
        "analisis": None,
        "sugerencias": None,
        "metricas": None
    }
    
    # ETAPA 1: Validación de entrada
    print("🔍 Etapa 1: Validando código de entrada...")
    es_valido, error = validar_codigo_python(codigo)
    if not es_valido:
        resultado["errores"].append(f"Validación fallida: {error}")
        return resultado
    print("✅ Código válido")
    
    # ETAPA 2: Análisis del código
    print("\n📊 Etapa 2: Analizando código...")
    prompt_analisis = f"""
Analiza este código Python y retorna un JSON con:
- complejidad: "baja", "media" o "alta"
- lineas: número de líneas
- funciones: número de funciones
- posibles_bugs: lista de posibles problemas

Código:
```python
{codigo}
```

Responde SOLO con JSON válido, sin texto adicional.
"""
    
    respuesta_analisis = llamada_ia_con_retry(prompt_analisis)
    if not respuesta_analisis:
        resultado["errores"].append("Fallo en análisis de código")
        return resultado
    
    # Validar respuesta JSON
    es_json_valido, datos_analisis = validar_respuesta_json(respuesta_analisis)
    if not es_json_valido:
        resultado["errores"].append("Respuesta de análisis no es JSON válido")
        # Intentar continuar con texto plano
        resultado["analisis"] = {"texto": respuesta_analisis}
    else:
        resultado["analisis"] = datos_analisis
        print(f"✅ Análisis completado: Complejidad {datos_analisis.get('complejidad', 'N/A')}")
    
    # ETAPA 3: Generar sugerencias
    print("\n💡 Etapa 3: Generando sugerencias...")
    prompt_sugerencias = f"""
Basándote en este análisis:
{json.dumps(resultado["analisis"], indent=2, ensure_ascii=False)}

Y este código:
```python
{codigo}
```

Genera sugerencias de mejora. Retorna JSON con:
- sugerencias: lista de sugerencias con "tipo" y "descripcion"
- prioridad: "alta", "media" o "baja" para cada sugerencia

Responde SOLO con JSON válido.
"""
    
    respuesta_sugerencias = llamada_ia_con_retry(prompt_sugerencias)
    if respuesta_sugerencias:
        es_json_valido, datos_sugerencias = validar_respuesta_json(respuesta_sugerencias)
        if es_json_valido:
            resultado["sugerencias"] = datos_sugerencias
            print(f"✅ {len(datos_sugerencias.get('sugerencias', []))} sugerencias generadas")
        else:
            resultado["sugerencias"] = {"texto": respuesta_sugerencias}
    
    # ETAPA 4: Calcular métricas
    print("\n📈 Etapa 4: Calculando métricas...")
    lineas = len(codigo.split('\n'))
    funciones = codigo.count('def ')
    clases = codigo.count('class ')
    
    resultado["metricas"] = {
        "lineas_codigo": lineas,
        "numero_funciones": funciones,
        "numero_clases": clases,
        "complejidad_ciclomatica_estimada": funciones * 2  # Estimación simple
    }
    print("✅ Métricas calculadas")
    
    resultado["exitoso"] = True
    return resultado

# Código de ejemplo
codigo_ejemplo = """
def calcular_factorial(n):
    if n < 0:
        raise ValueError("n debe ser positivo")
    if n == 0 or n == 1:
        return 1
    resultado = 1
    for i in range(2, n + 1):
        resultado *= i
    return resultado

def es_primo(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True
"""

# Ejecutar workflow
print("=" * 60)
print("WORKFLOW DE ANÁLISIS DE CÓDIGO")
print("=" * 60 + "\n")

resultado_final = workflow_analisis_codigo(codigo_ejemplo)

print("\n" + "=" * 60)
print("RESULTADO FINAL")
print("=" * 60)
print(f"\n✅ Exitoso: {resultado_final['exitoso']}")
if resultado_final['errores']:
    print(f"\n⚠️ Errores: {resultado_final['errores']}")
print(f"\n📊 Análisis:\n{json.dumps(resultado_final['analisis'], indent=2, ensure_ascii=False)}")
print(f"\n💡 Sugerencias:\n{json.dumps(resultado_final['sugerencias'], indent=2, ensure_ascii=False)}")
print(f"\n📈 Métricas:\n{json.dumps(resultado_final['metricas'], indent=2, ensure_ascii=False)}")

In [ ]:
# ESPACIO PARA TU CÓDIGO
def workflow_documentacion(codigo: str) -> Dict[str, Any]:
    """
    Workflow completo de generación de documentación con validaciones
    """
    resultado = {
        "exitoso": False,
        "errores": [],
        "documentacion": None
    }
    
    # TODO: Implementa las 5 etapas del workflow
    # Etapa 1: Validar código
    # Etapa 2: Generar documentación
    # Etapa 3: Validar documentación
    # Etapa 4: Generar índice
    # Etapa 5: Consolidar
    
    return resultado

# Prueba el workflow
# codigo_test = """
# def suma(a, b):
#     return a + b
# 
# class Calculadora:
#     def multiplicar(self, x, y):
#         return x * y
# """
# 
# resultado = workflow_documentacion(codigo_test)
# print(json.dumps(resultado, indent=2, ensure_ascii=False))

## 🔥 Ejercicio de Desafío 2.3 (Opcional)

Crea un sistema de code review automatizado con workflow completo:

1. **Análisis estático**: Detectar problemas de código
2. **Análisis de seguridad**: Buscar vulnerabilidades
3. **Análisis de performance**: Identificar cuellos de botella
4. **Generación de reporte**: Consolidar todos los análisis
5. **Sistema de scoring**: Asignar puntuación y prioridad

**Desafío extra**: Implementa un sistema de caché para evitar re-analizar código idéntico.

---

# 📝 Reflexión y Mejores Prácticas

## ✅ ¿Qué Aprendimos?

### 1. Agentes Multi-Paso
- **Planificación**: Descomponer tareas complejas mejora resultados
- **Secuencialidad**: Cada paso puede usar información de pasos anteriores
- **Modularidad**: Separar responsabilidades facilita mantenimiento

### 2. Function Calling
- **Extensibilidad**: Conectar IA con sistemas externos amplía capacidades
- **Datos en tiempo real**: Acceder a información actualizada
- **Integración**: Conectar con bases de datos, APIs, servicios

### 3. Workflows Robustos
- **Validación**: Verificar entrada y salida en cada etapa
- **Manejo de errores**: Fallbacks y recuperación graceful
- **Retry logic**: Reintentar operaciones fallidas
- **Logging**: Registrar cada paso para debugging

## ⚠️ Errores Comunes a Evitar

1. **Falta de validación**
   - ❌ Confiar ciegamente en respuestas del modelo
   - ✅ Validar entrada y salida en cada etapa

2. **Sin manejo de errores**
   - ❌ Asumir que todo funcionará siempre
   - ✅ Implementar try-catch y fallbacks

3. **Workflows demasiado complejos**
   - ❌ 20 pasos sin estructura clara
   - ✅ Máximo 5-7 pasos bien definidos

4. **Sin optimización de costos**
   - ❌ Llamadas innecesarias a modelos costosos
   - ✅ Usar modelos apropiados para cada tarea

5. **Falta de logging**
   - ❌ No registrar qué pasó en cada etapa
   - ✅ Logging detallado para debugging

## 🚀 Tips para Producción

1. **Monitoreo**: Trackea costos, latencia y errores
2. **Caché**: Almacena resultados de operaciones costosas
3. **Rate limiting**: Controla frecuencia de llamadas
4. **Circuit breakers**: Detén workflows si hay muchos errores
5. **Versionado**: Mantén versiones de prompts y workflows
6. **Testing**: Prueba cada etapa independientemente
7. **Documentación**: Documenta decisiones de diseño

## 📚 Recursos Adicionales

- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [Best Practices for AI Systems](https://platform.openai.com/docs/guides/production-best-practices)

---

## 🎓 Próximos Pasos

En el **Módulo 3** aprenderás:
- Context Engineering y RAG
- Embeddings y búsqueda semántica
- Integración de múltiples fuentes de contexto
- Sistemas completos de knowledge retrieval

¡Felicitaciones por completar el Módulo 2! 🎉